In [ ]:
import random
import numpy as np  
import torch
import matplotlib.pyplot as plt
from functools import partial
from os.path import dirname, join as pjoin
import scipy
import scipy.io as sio

from accuracy_bounds.data.generator_functions import random_uni_points_in_ball
from accuracy_bounds.utils.visualizations import plot_wckersize_conv, plot_avkersize_conv, visualize_ball_3d
from accuracy_bounds.algo.kernel_size import compute_wc_kernel_size, compute_av_kernel_size
from accuracy_bounds.algo.projections import apply_forwardmodel
from accuracy_bounds.data.dataset import GeneratorDataset
from accuracy_bounds.utils.utils import set_seed

### **Example 2:** Worst Case Kernel Size over Flouresence Microscopy Data

In [ ]:
# Get the filename for an example .mat file from the  examples/flourescencemicroscopy directory.
data_dir = './example_data/flourescencemicroscopy'

### Construct Feasible Sets for 1 Example of Flouresence Microscopy Data (I=1)

In [ ]:
def load_xy_components(mat_contents):
    """ Load xyz components per example, i.e. one measurement w, from matrix.
    Args:
        -mat_contents, dictionary
    Returns: 
        -feas: xyz component feasible set for measurement of matrix."""

    x = np.array(mat_contents['x_grnd']).flatten()
    xm=np.array(mat_contents['x_mcmc']).flatten()
    x=np.concatenate((x,xm), axis=None)

    y = np.array(mat_contents['y_grnd']).flatten()
    ym=np.array(mat_contents['y_mcmc']).flatten()
    y=np.concatenate((y,ym), axis=None)

    # z is zero currently
    #z = np.array(mat_contents['z_grnd']).flatten()
    #zm=np.array(mat_contents['z_mcmc']).flatten()
    #z=np.concatenate((z,zm), axis=None)

    #create feasible set for one measurement
    f_xy= np.stack((x,y), axis=1)
    return f_xy

In [ ]:
def load_ch_components(mat_contents):
    """ Load ch components per example, i.e. one measurement w, from matrix.
    Args:
        -mat_contents, dictionary
    Returns: 
        -feas: ch component feasible set for measurement of matrix."""

    c = np.array(mat_contents['C_grnd']).flatten()
    cm=np.array(mat_contents['C_mcmc']).flatten()
    c=np.concatenate((c,cm), axis=None)

    h = np.array(mat_contents['h_grnd']).flatten()
    hm=np.array(mat_contents['h_mcmc']).flatten()
    h=np.concatenate((h,hm), axis=None)

    #create feasible set for example 001
    f_ch= np.stack((c,h), axis=1)
    return f_ch

In [ ]:
# get list of xy feasible set data for each measurement, i.e. one example_00x.mat file corresponds to one measurement
feasible_sets_list = []
for i in range(1,6):      
  mat = pjoin(data_dir, f"example_00{i}.mat")
  # Load the .mat file contents
  mat_contents = sio.loadmat(mat)
  # get xy feasible set from example i
  f_xy=load_xy_components(mat_contents)
  # add feasible set to list of feasible sets
  feasible_sets_list.append(f_xy)
  print(f_xy)


### **Example 2:** Kernel Sizes for Example Flouresence Microscopy Data

In [ ]:
def compute_wc_av_diams_feas(feas):
    """" Compute average and worst case diams with the l2 norm for one feasible set.
    Args:
        -feas: array of elements in one feasible set
    Returns: 
        - diameter_mean_y: average diameter of feas (feasible set)
        - num_feas: number of elements in feas (feasible set)
        - max_diam_Fy: largest diameter of feas (feasible set)
    """
    max_diam_Fy = 0
    diameter_mean_y = 0
    diam_y = []
    num_feas = np.array(np.shape(feas))[0].astype(int)
    # compute diameters
    for h in range(0,num_feas,1):
        for j in range(0,h+1,1):
            #compute vectors in null space of F and their norm
            dist_ns = feas[h]-feas[j]
            diameter= np.linalg.norm(dist_ns, ord = 2)

            #add to diam_y 
            diam_y.append(diameter)

            #get ascending diams
            if diameter > max_diam_Fy:
                max_diam_Fy = diameter
                            
    # get mean over diams, with factor 2 due to symmetry of the norm of the compute vectors in null space of F (norm(x-z)=norm(z-x))
    # and divided by num_feas^2 ad we have that many terms
    if num_feas > 0:      
        # compute 2 times sum over diams to the power p divided by num_feas^2
        diameter_mean_y = 2*np.divide(np.sum(np.power(diam_y,2)), np.power(num_feas,2))
    elif num_feas==0:
        diameter_mean_y = 0  

    return diameter_mean_y, num_feas, max_diam_Fy
    

### Compute worst case and average kernel size per example

In [ ]:
for i in range(0,5):
    print(compute_wc_av_diams_feas(feasible_sets_list[i]))

This means that $2\sqrt{Kersize(F,\mathcal{M}_1,\mathcal{E},2)_1}= \frac{1}{N} \sqrt{\sum_{h,l=1}^{n} \| \Theta_l-\Theta_h \|^2}$, has the following values
$$001:  1.4942e-11 m $$
$$002:  6.8085e-12 m $$


### Compute  worst case and average kernel size over all examples

In [ ]:

def av_kernelsize(feasible_sets_list, p=2):
    """
    Computes the average kernel size for a noisy inverse problem under Algorithm 2.

    Args:
        - feasible_sets_list: list of feasible sets
        - p: order of the norm, default p=2 for MSE computation.

    Returns:
        Approximate average kernel size for for a set of input data samples.
    """

    av_kersizep = 0
    num_samples = len(feasible_sets_list)

    for i in range(0,num_samples):
        # compute diameter of feasible set for one input data point (num_feas will be used for statistics later on)
        diameter_mean_y, num_feas, max_diam_Fy = compute_wc_av_diams_feas(feasible_sets_list[i])
        #add diameters means for obtaining average kersize to the power p
        av_kersizep = av_kersizep + diameter_mean_y
        
    # get mean over input data
    av_kersize = np.divide(av_kersizep, num_samples)    
    # take power 1/p to obtain average kersize
    av_kersize =  np.power(av_kersize, 1/p)
    
    return av_kersize


In [ ]:
for i in range(0,5):
    print(compute_wc_av_diams_feas(feasible_sets_list[i]))

In [ ]:
print(av_kernelsize(feasible_sets_list, p=2))

This means for $I=5$, $$2\sqrt{Kersize(F,\mathcal{M}_1,\mathcal{E},2)_5}= 0.002628e-6m$$.

In [ ]:
def wc_kernelsize(feasible_sets_list, p=2):
    """
    Computes the worst-case kernel size for noisy inverse problem using Algorithm 1.

    Args:
        - feasible_sets_list: list of feasible sets
        - p: order of the norm, default p=2 for MSE computation.

    Returns:
        Approximate worst-case kernel size for a set of input data samples.
    """
    wc_kersize =0

    num_samples = len(feasible_sets_list)

    for i in range(0,num_samples):
        # compute diameter of feasible set for one input data point
        diameter_mean_y, num_feas, max_diam_Fy = compute_wc_av_diams_feas(feasible_sets_list[i])
        if max_diam_Fy > wc_kersize:
            wc_kersize = max_diam_Fy
    
    return wc_kersize

In [ ]:
print(wc_kernelsize(feasible_sets_list, p=2))

# TODOS: use visualization tool for visualizing x y z positions per example (i.e. measurement)
# test code

#### Collect Data

In [ ]:
target_data = random_uni_points_in_ball(num_points=num_points, radius=radius+epsilon, center=center, dim=3)                       
visualize_ball_3d(points=target_data, radius=radius, center=center, title="Target Data Points")

### Setup Linear Forward Operator and analytical Kernel Size

In [ ]:
# Toy forward operator
A = np.diag([1, 1, 0])  # Transformation matrix

#analytical wc kersize for ball around 0 with radius r+epsilon
wc_kernel_size_analytical = 2*radius+2*epsilon
#analytical av kersize for ball around 0 with radius r+epsilon sampled with uniform distribution
av_kernel_size_analytical = np.power((radius+epsilon),1/3)

# Distance measure
p=2
q=2

input_data = apply_forwardmodel(A, target_data)

In [ ]:
visualize_ball_3d(points=input_data, radius=radius, center=center, title="Inputs Data Points (Z axis is 0)")

### Setup WC-Kernal Size Computation

In [ ]:
max_k = 300
input_target_pairs = 500

##### Compute Worst-Case Kernel Size

In [ ]:
wc_kersizes = compute_wc_kernel_size(A, input_data[:input_target_pairs], target_data[:input_target_pairs], p, q, epsilon, max_k)
wc_kersize_final = wc_kersizes[-1]

plot_wckersize_conv(wc_kersize_final, wc_kersizes, wc_kernel_size_analytical, max_k)


### Setup AV-Kernal Size Computation

In [ ]:
max_k = 150
input_target_pairs = 1000

In [ ]:

av_kersizes = compute_av_kernel_size(A, input_data[:input_target_pairs], target_data[:input_target_pairs], p, q, epsilon, max_k)

av_kersizef = av_kersizes[-1]

plot_avkersize_conv(av_kersizef, av_kersizes, av_kernel_size_analytical, max_k)